In [4]:
import numpy as np
import pandas as pd
import scipy.stats as stats

# 1달간 다운로드 증감량 계산
def incdec_calculate(df):
    for day in range(1, 30):
        df.loc[:, f"Change_{day+1}Day"] = df.loc[:, f"{day+1}Day"] - df.loc[:, f"{day}Day"]
    return df

# 1달간 다운로드 상승률 계산
def incdecRate_calculate(df):
    for day in range(1, 30):
        df[f"ChangeRate_{day+1}Day"] = (df[f"Change_{day+1}Day"] / df[f"{day}Day"]) * 100
    return df

# 최근 29일간 누적 다운로드 수 계산
def cumulative_changes(df):
    df['Cumulative_Recent_29Day'] = df.loc[:, '1Day':'29Day'].cumsum(axis=1).iloc[:, -1]
    return df

df = pd.read_csv('20240615_Daily_Download.csv')
df_incdec = incdec_calculate(df)
df_incdec_Rate = incdecRate_calculate(df_incdec) 

df_incdec_Rate_copy = df_incdec_Rate.copy()

# ChangeRate_xDay 패턴에 맞게 가중치를 동적으로 계산
weights = {
    f'ChangeRate_{day}Day': weight
    for day, weight in zip(
        range(2, 30),  # 2부터 29까지
        [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 15, 20, 25, 30, 35, 40, 50, 60, 71, 75, 79, 83, 87, 90, 93, 96, 98, 100]
    )
}

df_incdec_Rate_copy.loc[:, 'Change_2Day_to_29Day_WeightedSum'] = sum(
    df_incdec_Rate_copy[col] * weight for col, weight in weights.items()
) / sum(weights.values())

df_incdec = df_incdec_Rate_copy.sort_values(by='Change_2Day_to_29Day_WeightedSum', ascending=False)

df_incdec['inc_score'] = range(len(df_incdec), 0, -1)
days_columns = [f"{i+1}Day" for i in range(30)]

def filter_and_sort(df, days_columns, threshold, exclude_indexes=None):
    """
    필터링하고 정렬하는 함수
    - df: 원본 데이터프레임
    - days_columns: 조건을 적용할 컬럼들
    - threshold: 필터링할 최소 값
    - exclude_indexes: 제외할 인덱스들 (기본값 None)
    """
    if exclude_indexes is None:
        exclude_indexes = set()

    # 조건에 맞는 데이터 필터링
    filtered_df = df[df[days_columns].ge(threshold).all(axis=1)]
    
    # 제외할 인덱스가 있으면 제외
    filtered_df = filtered_df[~filtered_df.index.isin(exclude_indexes)]
    
    # inc_score 기준으로 정렬
    sorted_df = filtered_df.sort_values(by='inc_score', ascending=False)
    
    return sorted_df, filtered_df.index

# 각 threshold 값에 대해 반복적으로 호출
ge_10000_df_cum, ge_10000_idx = filter_and_sort(df_incdec, days_columns, 10000)
ge_1000_df_cum, ge_1000_idx = filter_and_sort(df_incdec, days_columns, 1000, exclude_indexes=ge_10000_idx)
ge_100_df_cum, ge_100_idx = filter_and_sort(df_incdec, days_columns, 100, exclude_indexes=ge_10000_idx.union(ge_1000_idx))
ge_50_df_cum, ge_50_idx = filter_and_sort(df_incdec, days_columns, 50, exclude_indexes=ge_10000_idx.union(ge_1000_idx).union(ge_100_idx))
ge_10_df_cum, ge_10_idx = filter_and_sort(df_incdec, days_columns, 10, exclude_indexes=ge_10000_idx.union(ge_1000_idx).union(ge_100_idx).union(ge_50_idx))
ge_0_df_cum, ge_0_idx = filter_and_sort(df_incdec, days_columns, 0, exclude_indexes=ge_10000_idx.union(ge_1000_idx).union(ge_100_idx).union(ge_50_idx).union(ge_10_idx))

dfs = [ge_10000_df_cum, ge_1000_df_cum, ge_100_df_cum, ge_50_df_cum, ge_10_df_cum, ge_0_df_cum]

merged_df = pd.concat(dfs, ignore_index=True)
merged_df['inc_sort_score'] = range(len(merged_df), 0, -1)

merged_df_copy = merged_df.copy()
df_incdec_WeightSum_cumulative = cumulative_changes(merged_df_copy)
sorted_df = df_incdec_WeightSum_cumulative.sort_values(by='Cumulative_Recent_29Day', ascending=False)
sorted_df['sum_score'] = range(len(sorted_df), 0, -1)

def filter_and_sort_exclude(df, days_columns, threshold, exclude_indexes=None, sort_column='Cumulative_Recent_29Day'):
    """
    특정 기준에 맞는 데이터를 필터링하고, 정렬한 후
    제외할 인덱스를 제외한 새로운 데이터프레임을 반환하는 함수.
    
    - df: 원본 데이터프레임
    - days_columns: 필터링할 컬럼들
    - threshold: 필터링할 최소 값
    - exclude_indexes: 제외할 인덱스들 (기본값 None)
    - sort_column: 정렬할 컬럼명 (기본값 'Cumulative_Recent_29Day')
    """
    if exclude_indexes is None:
        exclude_indexes = set()

    # 조건에 맞는 데이터 필터링
    filtered_df = df[df[days_columns].ge(threshold).all(axis=1)]
    
    # 제외할 인덱스가 있으면 제외
    filtered_df = filtered_df[~filtered_df.index.isin(exclude_indexes)]
    
    # 정렬
    sorted_df = filtered_df.sort_values(by=sort_column, ascending=False)
    
    return sorted_df, filtered_df.index

# 각 threshold 값에 대해 반복적으로 호출
ge_10000_df_cum, ge_10000_idx = filter_and_sort_exclude(sorted_df, days_columns, 10000)
ge_1000_df_cum, ge_1000_idx = filter_and_sort_exclude(sorted_df, days_columns, 1000, exclude_indexes=ge_10000_idx)
ge_100_df_cum, ge_100_idx = filter_and_sort_exclude(sorted_df, days_columns, 100, exclude_indexes=ge_10000_idx.union(ge_1000_idx))
ge_50_df_cum, ge_50_idx = filter_and_sort_exclude(sorted_df, days_columns, 50, exclude_indexes=ge_10000_idx.union(ge_1000_idx).union(ge_100_idx))
ge_10_df_cum, ge_10_idx = filter_and_sort_exclude(sorted_df, days_columns, 10, exclude_indexes=ge_10000_idx.union(ge_1000_idx).union(ge_100_idx).union(ge_50_idx))
ge_0_df_cum, ge_0_idx = filter_and_sort_exclude(sorted_df, days_columns, 0, exclude_indexes=ge_10000_idx.union(ge_1000_idx).union(ge_100_idx).union(ge_50_idx).union(ge_10_idx))

len_ge_10000_df = len(ge_10000_df_cum)
len_ge_1000_df = len(ge_1000_df_cum)
len_ge_100_df = len(ge_100_df_cum)
len_ge_50_df = len(ge_50_df_cum)
len_ge_10_df = len(ge_10_df_cum)
len_ge_0_df = len(ge_0_df_cum)

print(f"Length of ge_10000_df: {len_ge_10000_df}")
print(f"Length of ge_1000_df: {len_ge_1000_df}")
print(f"Length of ge_100_df: {len_ge_100_df}")
print(f"Length of ge_50_df: {len_ge_50_df}")
print(f"Length of ge_10_df: {len_ge_10_df}")
print(f"Length of ge_0_df: {len_ge_0_df}")

# 병합을 위한 데이터프레임 리스트
dfs = [ge_10000_df_cum, ge_1000_df_cum, ge_100_df_cum, ge_50_df_cum, ge_10_df_cum, ge_0_df_cum]

# 모든 데이터프레임을 병합
merged_df = pd.concat(dfs, ignore_index=True)

merged_df['consistency_score'] = range(len(merged_df), 0, -1)
merged_df = merged_df.sort_values(by='Cumulative_Recent_29Day', ascending=False)

sum_score_list = merged_df['sum_score'].to_list()
inc_score_list = merged_df['inc_sort_score'].to_list()
consistency_score_list = merged_df['consistency_score'].to_list()

data = {
    'Total_Downloads_Rank': sum_score_list,
    'Growth_Rate_Rank': inc_score_list,
    'Consistency_Rank': consistency_score_list
}
df = pd.DataFrame(data)

# 스피어만 순위 상관계수 계산
spearman_corr_total_growth, p_value_total_growth = stats.spearmanr(df['Total_Downloads_Rank'], df['Growth_Rate_Rank'])
spearman_corr_total_consistency, p_value_total_consistency = stats.spearmanr(df['Total_Downloads_Rank'], df['Consistency_Rank'])
spearman_corr_growth_consistency, p_value_growth_consistency = stats.spearmanr(df['Growth_Rate_Rank'], df['Consistency_Rank'])

# 결과 출력
print(f"Spearman correlation between Total Downloads and Growth Rate: {spearman_corr_total_growth}, P-value: {p_value_total_growth}")
print(f"Spearman correlation between Total Downloads and Consistency: {spearman_corr_total_consistency}, P-value: {p_value_total_consistency}")
print(f"Spearman correlation between Growth Rate and Consistency: {spearman_corr_growth_consistency}, P-value: {p_value_growth_consistency}")

merged_df['row_mean'] = merged_df[['inc_sort_score', 'sum_score', 'consistency_score']].mean(axis=1)
mean_df = merged_df.sort_values(by='row_mean', ascending=False)
mean_df.to_csv('20240615_모델_순위.csv')

Length of ge_10000_df: 84
Length of ge_1000_df: 305
Length of ge_100_df: 854
Length of ge_50_df: 465
Length of ge_10_df: 2906
Length of ge_0_df: 5386
Spearman correlation between Total Downloads and Growth Rate: 0.6687644090556442, P-value: 0.0
Spearman correlation between Total Downloads and Consistency: 0.894415291204153, P-value: 0.0
Spearman correlation between Growth Rate and Consistency: 0.8658307077063071, P-value: 0.0
